<img width="10%" alt="Naas" src="https://landen.imgix.net/jtci2pxwjczr/assets/5ice39g4.png?w=160"/>

# Revenue & COGS by Segment Dashboard 

**Tags:** #dashboard #plotly #dash #naas #asset #automation #ai #analytics

**Author:** [Jeremy Ravenel](https://www.linkedin.com/in/jeremyravenel/) & [Fernando Chavez Osuna](https://www.linkedin.com/in/fernando-chavez-osuna-1a420a181)

This notebook enables you to generate a dashboard about Revenue & COGS by segment.

## Input

### Install packages

In [2]:
#!pip install dash --user   
#!pip install dash-html-components                                         
#!pip install dash-core-components                                     
#!pip install plotly
!pip install dash_bootstrap_components --user

  Using cached dash_bootstrap_components-1.2.1-py3-none-any.whl (216 kB)

[notice] A new release of pip available: 22.2.1 -> 22.2.2
[notice] To update, run: pip install --upgrade pip


### Import library

In [1]:
import dash
from dash import html, dcc, Input, Output, State
import dash_bootstrap_components as dbc
import plotly.graph_objects as go
import plotly.express as px
import os
import naas_drivers
from dash_bootstrap_components._components.Container import Container
import pandas as pd

### Defining the port of the dashboard

In [2]:
DASH_PORT = 8050

## Model

### Data

#### Input data from Gsheets

In [9]:
spreadsheet_id = "1yi0qzuUEnE9wMWWQFIVq5Uou8ChxFqF0MMqFM5FtVIM"

#### Highlighted KPIs

In [11]:
# Dataframe is returned
df_hkpis = naas_drivers.gsheet.connect(spreadsheet_id).get(
    sheet_name="KPIS"
)

#### Mock data for bar chart

In [3]:
df = pd.DataFrame({
    "Segment": ["Channel Part", "Channel Part", 
                "Enterprise", "Enterprise", 
                "Goverment", "Goverment", 
                "Midmarket", "Midmarket", 
                "Small Business", "Small Business"],
    "Gross_Sales_COGS": [2000000, 1000000, 
                         20000000, 18000000, 
                         58000000, 40000000, 
                         3000000, 3000000, 
                         50000000, 38000000],
    "Measures": ["Gross Sales", "COGS", 
                 "Gross Sales","COGS", 
                 "Gross Sales", "COGS", 
                 "Gross Sales", "COGS", 
                 "Gross Sales", "COGS"]
})

### Create Graphs

#### Gross Sales & COGS by Segment

In [4]:
def sales_cogs_segment(df, segment, gross_sales, measures):
    
    bar_fig = px.bar(df, 
                     x=segment, 
                     y=gross_sales, 
                     color=measures, 
                     barmode="group", 
                     title='<b>Gross Sales and COGS by Segment', 
                     color_discrete_sequence=px.colors.qualitative.T10
                    )
    
    bar_fig.update_xaxes(ticks='outside', title='Segment')
    bar_fig.update_yaxes(ticks='outside', title='Gross Sales & COGS')
    
    bar_fig.update_layout(plot_bgcolor='rgba(0,0,0,0)', width=600, height=700)
    
    return bar_fig  

#### Sales by Segment

In [5]:
def sales_segment(df, segment, sales):
    pie_fig = px.pie(df, values=sales, names=segment, title='<b>Sales by Segment', hole=0.5, width=400, height=400)
    
    pie_fig.update_traces(textinfo='none')
    
    return pie_fig  

#### Gross Sales by Country and Product

In [6]:
new_df = pd.DataFrame({
            "country": ["Mexico","Mexico","Mexico","Mexico","Mexico","Mexico", 
                        "Japan","Japan","Japan","Japan","Japan","Japan", 
                        "Germany","Germany","Germany","Germany","Germany","Germany", 
                        "Austria", "Austria","Austria","Austria","Austria","Austria",
                        "S.Korea","S.Korea","S.Korea","S.Korea","S.Korea","S.Korea"],
            "sales": [4000000, 1000000, 2000000, 2000000, 4000000, 5000000,
                      3000000, 1500000, 1000000, 2500000, 3400000, 4500000,
                      2300000, 1200000, 900000, 1500000, 2500000, 3200000,
                      2000000, 1000000, 700000, 1100000, 2000000, 2500000,
                      1500000, 900000, 900000, 1600000, 1700000, 2000000,],
            "product": ["Velo", "VTT", "Paseo", "Montana", "Carretera", "Amarilla",
                        "Velo", "VTT", "Paseo", "Montana", "Carretera", "Amarilla",
                        "Velo", "VTT", "Paseo", "Montana", "Carretera", "Amarilla",
                        "Velo", "VTT", "Paseo", "Montana", "Carretera", "Amarilla",
                        "Velo", "VTT", "Paseo", "Montana", "Carretera", "Amarilla",]
})
new_df = new_df.groupby(['country','product'])['sales'].sum().reset_index()
new_df = new_df.pivot(index='country', columns='product')['sales'].fillna(0)

def sales_country_product(df):
    heat_fig = px.imshow(df,
                         labels=dict(x="<b>Product", y="<b>Country", color="<b>Gross Sales"),
                         x=df.columns, 
                         y=df.index, 
                         title='<b>Gross Sales by Country and Product', 
                         width=500, height=600,
                         aspect="auto")
    
    return heat_fig  

### Design

#### Initialize Dash app

In [7]:
app = dash.Dash(requests_pathname_prefix=f'/user/{os.environ.get("JUPYTERHUB_USER")}/proxy/{DASH_PORT}/', 
                external_stylesheets=[dbc.themes.BOOTSTRAP])   
#app = dash.Dash() if you are not in Naas

#### Search Bar

In [ ]:
search_bar = dbc.Row(
    [
        dbc.Col(dbc.Input(type="search", placeholder="Explore")),
        dbc.Col(
            dbc.Button(
                "Search", color="primary", className="ms-2", n_clicks=0
            ),
            width="auto",
        ),
    ],
    className="g-0 ms-auto flex-nowrap mt-3 mt-md-0",
    align="center",
)

#### Dropdown Menu

In [ ]:
nav_dropdown = dbc.DropdownMenu(
                    children=[
                        dbc.DropdownMenuItem("By Country", header=True),
                        dbc.DropdownMenuItem("France", href="#"),
                        dbc.DropdownMenuItem("USA", href="#"),
                        dbc.DropdownMenuItem("By Topic", header=True),
                        dbc.DropdownMenuItem("Temperature anomaly", href="#"),
                        dbc.DropdownMenuItem("COVID 19", href="#"),
                    ],
                    label="Explore more",
                ),

#### Create app layout (Working)

In [10]:
app.layout = html.Div(
    [
        dbc.Navbar(
            dbc.Container(
                [
                    html.A(
                        # Use row and col to control vertical alignment of logo / brand
                        dbc.Row(
                            [
                                dbc.Col(html.Img(src="https://pbs.twimg.com/profile_images/1243203211114274817/WqWaa9Bm_400x400.jpg", height="30px")),
                                dbc.Col(dbc.NavbarBrand("Revenue & COGS by Segment", className="ms-2")),
                            ],
                            align="center",
                            className="g-0",
                        ),
                        href="https://mobile.twitter.com/ws_room/photo",
                        style={"textDecoration": "none"},
                    ),
                    #dbc.NavbarToggler(id="navbar-toggler", n_clicks=0),
                    dbc.NavItem(dbc.NavLink("Twitter", href="https://mobile.twitter.com/ws_room",target="_blank")),
                    dbc.DropdownMenu(
                        children=[
                            dbc.DropdownMenuItem("By Country", header=True),
                            dbc.DropdownMenuItem("France", href="#"),
                            dbc.DropdownMenuItem("USA", href="#"),
                            dbc.DropdownMenuItem("By Topic", header=True),
                            dbc.DropdownMenuItem("Temperature anomaly", href="#"),
                            dbc.DropdownMenuItem("COVID 19", href="#"),
                        ],
                        nav=True,
                        in_navbar=True,
                        label="Explore more",)
                ]
            ),
            color="dark",
            dark=True,
        ),        
        
        #Charts
        dbc.Container([ 
            dbc.Row([
               dbc.Col([
                    dbc.Row([
                        dbc.Col(
                            dcc.Graph(
                                id='sales-cogs-by-segment',
                                figure=sales_cogs_segment(df, df.Segment, df.Gross_Sales_COGS, df.Measures))
                        ),
                    ]),

               ]),
               dbc.Col([
                    dbc.Row([
                        dbc.Col([
                            dcc.Graph(
                                id='sales-by-segment',
                                figure=sales_segment(df, df.Segment, df.Gross_Sales_COGS))
                        ]),
                    ]),
                    dbc.Row([
                        dbc.Col([
                           dcc.Graph(
                                id='heatmap',
                                figure=sales_country_product(new_df)) 
                        ]),
                    ]),
               ]), 
            ])
        ])           

    ]
)

# add callback for toggling the collapse on small screens
@app.callback(
    Output("navbar-collapse", "is_open"),
    [Input("navbar-toggler", "n_clicks")],
    [State("navbar-collapse", "is_open")],
)
def toggle_navbar_collapse(n, is_open):
    if n:
        return not is_open
    return is_open

## Output

### Generate URL and show logs

In [ ]:
if __name__ == '__main__':
    app.run_server(proxy=f"http://127.0.0.1:{DASH_PORT}::https://app.naas.ai")

Dash is running on https://app.naas.ai/user/fcha56@gmail.com/proxy/8050/

 * Serving Flask app '__main__'
 * Debug mode: off


 * Running on http://127.0.0.1:8050 (Press CTRL+C to quit)
127.0.0.1 - - [12/Aug/2022 11:10:07] "GET / HTTP/1.1" 200 -
127.0.0.1 - - [12/Aug/2022 11:10:07] "GET /_dash-component-suites/dash_bootstrap_components/_components/dash_bootstrap_components.v1_2_1m1660292126.min.js HTTP/1.1" 200 -
127.0.0.1 - - [12/Aug/2022 11:10:07] "GET /_dash-dependencies HTTP/1.1" 200 -
127.0.0.1 - - [12/Aug/2022 11:10:07] "GET /_dash-layout HTTP/1.1" 200 -
127.0.0.1 - - [12/Aug/2022 11:10:08] "GET /_dash-component-suites/dash/dcc/async-graph.js HTTP/1.1" 200 -
127.0.0.1 - - [12/Aug/2022 11:10:08] "GET /_dash-component-suites/dash/dcc/async-plotlyjs.js HTTP/1.1" 200 -
127.0.0.1 - - [12/Aug/2022 11:10:08] "GET /_favicon.ico?v=2.6.1 HTTP/1.1" 200 -
127.0.0.1 - - [12/Aug/2022 11:17:02] "GET /_dash-component-suites/dash/dcc/dash_core_components-shared.js.map HTTP/1.1" 200 -
127.0.0.1 - - [12/Aug/2022 11:17:02] "GET /_dash-component-suites/dash/dcc/async-graph.js.map HTTP/1.1" 200 -
127.0.0.1 - - [12/Aug/2022 11